# HyperStream Tutorial 1: Introduction

## Requirements

In order to run this and the following tutorials, it is necessary to have access to a MongoDB server running in the **localhost port 27017**. It is possible to change the host and port of the MongoDB server by modifying the configuration file __hyperstream_config.json__ located in the same folder as this notebook.

In [1]:
import sys
sys.path.append("../") # Add parent dir in the Path

from hyperstream import HyperStream

## Starting an Hyperstream instance

First of all, we will create a HyperStream instance. This instance will connect to the MongoDB server that is specified in the configuration file and it will raise an error if the MongoDB server is not running.

In [6]:
hs = HyperStream()
print hs

HyperStream version 0.2.6, connected to mongodb://localhost:27017/hyperstream


## Memory and tools



In [ ]:
from hyperstream.workflow.workflow import Workflow

from hyperstream import StreamId, TimeInterval
from pytz import UTC
from datetime import datetime, timedelta

In [3]:
M = hs.channel_manager.memory
T = hs.channel_manager.tools

In [4]:
clock_sid = StreamId(name="clock")
clock_tool = T[clock_sid].window().last().value()

ticker = M.get_or_create_stream(stream_id=StreamId(name="ticker"))

now = datetime.utcnow().replace(tzinfo=UTC)
before = (now - timedelta(seconds=6)).replace(tzinfo=UTC)
ti = TimeInterval(before, now)

clock_tool.execute(sources=[], sink=ticker, interval=ti, alignment_stream=None)

In [5]:
for timestamp, value in ticker.window().items():
    print('[%s]: %s' % (timestamp, value))

[2017-06-15 10:40:23+00:00]: 2017-06-15 10:40:23+00:00
[2017-06-15 10:40:24+00:00]: 2017-06-15 10:40:24+00:00
[2017-06-15 10:40:25+00:00]: 2017-06-15 10:40:25+00:00
[2017-06-15 10:40:26+00:00]: 2017-06-15 10:40:26+00:00
[2017-06-15 10:40:27+00:00]: 2017-06-15 10:40:27+00:00
[2017-06-15 10:40:28+00:00]: 2017-06-15 10:40:28+00:00
